In [1]:
# define activation function
import math
import random

def sigmoid(x):
    return 1.0 / (1.0 + math.exp(-x))

def sigmoid_derivative(output):
    return output * (1.0 - output)

In [2]:
# initialise network parameters
def initialize_network(n_inputs, n_hidden, n_outputs):
    network = []

    hidden_layer = []
    for _ in range(n_hidden):
        neuron = {
            'weights': [random.uniform(-0.5, 0.5) for _ in range(n_inputs + 1)]
        }
        hidden_layer.append(neuron)
    network.append(hidden_layer)

    output_layer = []
    for _ in range(n_outputs):
        neuron = {
            'weights': [random.uniform(-0.5, 0.5) for _ in range(n_hidden + 1)]
        }
        output_layer.append(neuron)
    network.append(output_layer)

    return network

In [3]:
# Forward Propagation
def forward_propagate(network, row):
    inputs = row[:-1]
    for layer in network:
        new_inputs = []
        for neuron in layer:
            activation = neuron['weights'][0]  # bias
            for i in range(len(inputs)):
                activation += neuron['weights'][i + 1] * inputs[i]
            neuron['output'] = sigmoid(activation)
            new_inputs.append(neuron['output'])
        inputs = new_inputs
    return inputs


In [4]:
# Backward Propagation
def backward_propagate_error(network, expected):
    for i in reversed(range(len(network))):
        layer = network[i]
        errors = []

        if i == len(network) - 1:  # output layer
            for j, neuron in enumerate(layer):
                errors.append(expected[j] - neuron['output'])
        else:  # hidden layer
            for j in range(len(layer)):
                error = 0.0
                for neuron in network[i + 1]:
                    error += neuron['weights'][j + 1] * neuron['delta']
                errors.append(error)

        for j, neuron in enumerate(layer):
            neuron['delta'] = errors[j] * sigmoid_derivative(neuron['output'])

In [5]:
# weight update with sgd
def update_weights(network, row, l_rate):
    inputs = row[:-1]
    for i, layer in enumerate(network):
        if i != 0:
            inputs = [neuron['output'] for neuron in network[i - 1]]

        for neuron in layer:
            neuron['weights'][0] += l_rate * neuron['delta']  # bias
            for j in range(len(inputs)):
                neuron['weights'][j + 1] += l_rate * neuron['delta'] * inputs[j]


In [6]:
# Training with sgd
def train_network(network, train, l_rate, n_epoch):
    for epoch in range(n_epoch):
        sum_error = 0.0
        for row in train:
            outputs = forward_propagate(network, row)
            expected = [row[-1]]
            sum_error += (expected[0] - outputs[0]) ** 2
            backward_propagate_error(network, expected)
            update_weights(network, row, l_rate)
        print(f"Epoch={epoch}, Error={sum_error:.4f}")


In [7]:
def predict(network, row):
    output = forward_propagate(network, row)[0]
    return 1 if output >= 0.5 else 0


In [8]:
# dataset
dataset = [
    [1, 2, 0],
    [5, 4, 1],
    [2, 1, 0],
    [3, 3, 1],
    [7, 2, 1],
    [5, 2, 1],
    [6, 1, 1],
    [8, -0.2, 1],
    [7, 3, 1]
]

In [9]:
network = initialize_network(2, 2, 1)
train_network(network, dataset, l_rate=0.3, n_epoch=15)

for row in dataset:
    output = forward_propagate(network, row)[0]
    prediction = predict(network, row)
    print(f"Expected={row[-1]}, Output={output:.3f}, Predicted={prediction}")


Epoch=0, Error=2.2079
Epoch=1, Error=1.8593
Epoch=2, Error=1.6474
Epoch=3, Error=1.5381
Epoch=4, Error=1.4813
Epoch=5, Error=1.4489
Epoch=6, Error=1.4281
Epoch=7, Error=1.4131
Epoch=8, Error=1.4011
Epoch=9, Error=1.3906
Epoch=10, Error=1.3809
Epoch=11, Error=1.3715
Epoch=12, Error=1.3621
Epoch=13, Error=1.3527
Epoch=14, Error=1.3432
Expected=0, Output=0.685, Predicted=1
Expected=1, Output=0.783, Predicted=1
Expected=0, Output=0.718, Predicted=1
Expected=1, Output=0.750, Predicted=1
Expected=1, Output=0.799, Predicted=1
Expected=1, Output=0.785, Predicted=1
Expected=1, Output=0.790, Predicted=1
Expected=1, Output=0.797, Predicted=1
Expected=1, Output=0.800, Predicted=1
